<a href="https://colab.research.google.com/github/tomvangasse/game-store-manager/blob/main/tomsMenuProgram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
Upload files here
Click run and then click choose files:

    Video_Game_Info.txt
    Board_Game_Info.txt
    Subscription_Info.txt
    Game_feedback.txt
    Booking.txt
    Rental.txt
    feedbackManager.pyc
    subscriptionManager.pyc
'''


from google.colab import files
uploaded = files.upload()

Saving Board_Game_Info.txt to Board_Game_Info.txt
Saving Booking.txt to Booking.txt
Saving Game_Feedback.txt to Game_Feedback.txt
Saving Rental.txt to Rental.txt
Saving Subscription_Info.txt to Subscription_Info.txt
Saving Video_Game_Info.txt to Video_Game_Info.txt
Saving feedbackManager.pyc to feedbackManager.pyc
Saving subscriptionManager.pyc to subscriptionManager.pyc


In [ ]:
import subscriptionManager as smTV
import feedbackManager as fbTV

sub = smTV.load_subscriptions()

print(sub)


Register / login

In [3]:

def register(userID):
    import subscriptionManager as smTV
    import datetime
    from dateutil.relativedelta import relativedelta as rd

    print("Unregistered account, please register now:")
    while True:
        subscriptionType = input("Enter subscription type (Basic or premium): ").lower()
        if subscriptionType != "basic" and subscriptionType != "premium":
            print("Invalid subscription type")
        else:
            break
    date = datetime.datetime.now()
    startTime = date.strftime("%Y-%m-%d")
    endTime = (date + rd(years=3)).strftime("%Y-%m-%d")

    with open("Subscription_Info.txt", mode = "a") as file:
        file.write(f"\n{userID},{subscriptionType},{startTime},{endTime}")

    print(f"Logged in as {userID}")
    return subscriptionType


def login(userID):
    import subscriptionManager as smTV

    subscriptions = smTV.load_subscriptions()

    if smTV.check_subscription(userID, subscriptions):
        print(f"Logged in as {userID}")

    else:
        register(userID)

def checkUserLength(userID):
    if len(userID) != 4:
        return False
    else:
        return True

Game search

In [2]:
import csv
def getVideoDict():
    with open("Video_Game_Info.txt", mode="r", newline="") as file:
        reader = csv.DictReader(file)
        boardGameDict = {row["GameID"]: row for row in reader}
    return boardGameDict

def getBoardDict():
    with open("Board_Game_Info.txt", mode="r", newline="") as file:
        reader = csv.DictReader(file)
        boardGameDict = {row["GameID"]: row for row in reader}
    return boardGameDict

def printGames(gameType):
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    if gameType == "b":
        for i in boardDict:
            currentGame = boardDict[i]
            print(f"{currentGame["GameID"]}: {(currentGame["Game name"]): <25} Available / Unavailable: {currentGame["Available copies"]} / {currentGame["Unavailable copies"]}")
        print("\n")
    elif gameType == "v":
        for i in videoDict:
            currentGame = videoDict[i]
            print(f"{currentGame["GameID"]}: {(currentGame["Game name"]): <25} Available / Unavailable: {currentGame["Available copies"]} / {currentGame["Unavailable copies"]}")
        print("\n")
    else:
        return

def printGameInfo(GameID, gameType):
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    if gameType == "b":
        gameInfo = boardDict[GameID]
        for key, value in gameInfo.items():
            print(f"{key: >20}:  {value}")
        print("\n")
    elif gameType == "v":
        gameInfo = videoDict[GameID]
        for key, value in gameInfo.items():
            print(f"{key: >20}:  {value}")
        print("\n")
    else:
        return


def gameSearch():
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    gameType = input('''Board games (b)
Video games (v)
Quit (q)
''')

    if gameType == "q":
        return False

    gameSelection = input('''Print games (press enter)
Select a specific game (enter GameID)
''')
    gameSelection
    if gameSelection == "":
        printGames(gameType)
    elif gameSelection == "q":
        return False
    else:
        printGameInfo(gameSelection, gameType)

while True:
    if gameSearch() == False:
        break


Board games (b)
Video games (v)
Quit (q)
q


Game rent

In [6]:
import subscriptionManager as smTV
import feedbackManager as fbTV


def checkAvailability(gameID, gameType):
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    if gameType == "b":
        currentGame = boardDict[gameID]
        if currentGame["Available copies"] == "0":
            return False
        else:
            return True

    elif gameType == "v":
        currentGame = videoDict[gameID]
        if currentGame["Available copies"] == "0":
            return False
        else:
            return True
    else:
        return

def checkSubscription(userID):
    while True:
        subscriptions = smTV.load_subscriptions()
        if smTV.check_subscription(userID, subscriptions):
            break
        login(userID)

    subType = subscriptions[userID]["SubscriptionType"]

def rentGame():
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    while True:
        userID = input("Enter user ID: ")
        if checkUserLength(userID):
            checkSubscription(userID)
            break
        else:
            print("Invalid user ID")


    selectedGame = input('''Enter the GameID for the desired game:
''')
    if selectedGame in videoDict:
        print(checkAvailability(selectedGame, "v"))
    elif selectedGame in boardDict:
        print(checkAvailability(selectedGame, "b"))

    else:
        print("Invalid GameID")



#with open("Rental.txt", mode="a") as file:
    #file.write(f"\n{gameID},{Rental date}

rentGame()

Enter user ID: stve
Unregistered account, please register now:
Enter subscription type (Basic or premium): premium
Logged in as stve
Enter the GameID for the desired game:
chss01b
True


In [ ]:
import subscriptionManager as smTV
import feedbackManager as fbTV

sub = smTV.load_subscriptions()
ID = "tomv"
smTV.check_subscription(ID, sub)

True